In [79]:
# Import libraries and dependencies
import os
import pandas as pd
import alpaca_trade_api as tradeapi
import numpy as np

# Load .env environment variables
from dotenv import load_dotenv
load_dotenv()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [80]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2"
)

In [91]:
from os import getenv
from dotenv import load_dotenv
from alpaca_trade_api.rest import REST, TimeFrame, TimeFrameUnit
import pandas as pd
from datetime import date, timedelta

def get_bars_alpaca(start_date=str(date.today()-timedelta(days=365*5)),end_date=str(date.today()-timedelta(days=1)),ticker='SOYB'):
    load_dotenv()
    alpaca_api_key=getenv('ALPACA_API_KEY')
    secret_key=getenv('ALPACA_SECRET_KEY')
    if type(alpaca_api_key) == 0:
        raise Exception("No ALPACA API KEY was loaded. Please save a .env file containing an ALPACA_API_KEY in the local directory.")
    if type(secret_key) == 0:
        raise Exception("No ALPACA SECRET KEY was loaded. Please save a .env file containing an ALPACA_SECRET_KEY in the local directory.")
    api=REST(key_id=alpaca_api_key,secret_key=secret_key)
    df=api.get_bars("SOYB", TimeFrame.Day, start_date, end_date, adjustment='raw').df
    return(df)



data_df = get_bars_alpaca()
data_df

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2018-03-26 04:00:00+00:00,18.82,18.8802,18.7400,18.7752,29951,79,18.833189
2018-03-27 04:00:00+00:00,18.75,18.8100,18.6000,18.6100,14154,93,18.716162
2018-03-28 04:00:00+00:00,18.66,18.6600,18.5401,18.6000,14027,32,18.613011
2018-03-29 04:00:00+00:00,18.57,19.1124,18.5100,19.0500,51977,176,18.942040
2018-04-02 04:00:00+00:00,19.10,19.1400,18.8811,18.9400,12591,63,19.038768
...,...,...,...,...,...,...,...
2023-03-16 04:00:00+00:00,27.31,27.3900,27.1500,27.3400,22040,272,27.283226
2023-03-17 04:00:00+00:00,27.34,27.3400,27.0675,27.0718,23877,183,27.147992
2023-03-20 04:00:00+00:00,27.02,27.1405,26.8000,27.0900,21903,209,27.035517


In [92]:
data_df['test target'] = np.where(data_df['open'] < data_df['close'], 1, 0)

In [93]:
df = pd.DataFrame(data_df)
df.columns = ['Feature 1', 'Feature 2', 'Feature 3', 'Feature 4', 'Feature 5', 'Feature 6', 'Feature 7', 'target']
df

,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,target
timestamp,,,,,,,,
2018-03-26 04:00:00+00:00,18.82,18.8802,18.7400,18.7752,29951,79,18.833189,0
2018-03-27 04:00:00+00:00,18.75,18.8100,18.6000,18.6100,14154,93,18.716162,0
2018-03-28 04:00:00+00:00,18.66,18.6600,18.5401,18.6000,14027,32,18.613011,0
2018-03-29 04:00:00+00:00,18.57,19.1124,18.5100,19.0500,51977,176,18.942040,1
2018-04-02 04:00:00+00:00,19.10,19.1400,18.8811,18.9400,12591,63,19.038768,0
...,...,...,...,...,...,...,...,...
2023-03-16 04:00:00+00:00,27.31,27.3900,27.1500,27.3400,22040,272,27.283226,1
2023-03-17 04:00:00+00:00,27.34,27.3400,27.0675,27.0718,23877,183,27.147992,0
2023-03-20 04:00:00+00:00,27.02,27.1405,26.8000,27.0900,21903,209,27.035517,1


In [84]:
X = df.iloc[:, 0:7].values
y = df["target"].values


In [85]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_scaler = StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [94]:
neuron = Sequential()
number_inputs = 2
number_hidden_nodes = 3

neuron.add(Dense(units=number_hidden_nodes, input_shape=(X_train.shape[1],), activation="relu", input_dim=number_inputs))

number_classes = 1

neuron.add(Dense(units=number_classes, activation="leaky_relu"))
neuron.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 3)                 24        
                                                                 
 dense_13 (Dense)            (None, 1)                 4         
                                                                 
Total params: 28
Trainable params: 28
Non-trainable params: 0
_________________________________________________________________


In [95]:
neuron.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model = neuron.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
30/30 [==============================] - 0s 965us/step - loss: 6.3568 - accuracy: 0.4682
Epoch 2/50
30/30 [==============================] - 0s 997us/step - loss: 5.7454 - accuracy: 0.4703
Epoch 3/50
30/30 [==============================] - 0s 1ms/step - loss: 5.2007 - accuracy: 0.4703
Epoch 4/50
30/30 [==============================] - 0s 1ms/step - loss: 4.6154 - accuracy: 0.4692
Epoch 5/50
30/30 [==============================] - 0s 1ms/step - loss: 4.0323 - accuracy: 0.4735
Epoch 6/50
30/30 [==============================] - 0s 1ms/step - loss: 3.7047 - accuracy: 0.4735
Epoch 7/50
30/30 [==============================] - 0s 1ms/step - loss: 3.3789 - accuracy: 0.4724
Epoch 8/50
30/30 [==============================] - 0s 1ms/step - loss: 3.3467 - accuracy: 0.4841
Epoch 9/50
30/30 [==============================] - 0s 1ms/step - loss: 3.1711 - accuracy: 0.4820
Epoch 10/50
30/30 [==============================] - 0s 1ms/step - loss: 3.0091 - accuracy: 0.4809
Epoch 11/50
30/